<a href="https://colab.research.google.com/github/ienev4/Stocks-Public/blob/main/Yields.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import plotly.graph_objects as go

In [4]:
data=pd.read_excel('Yields Historically.xlsx',skiprows=6,usecols=[0,1,3,5,7],names=['Date','10Y','5Y','2Y','30Y'],index_col='Date',parse_dates=['Date'])
data=data.reindex(data.index.sort_values(ascending=True))

In [5]:
monthly_data=data.resample('M').last()

In [6]:
monthly_data.isna().sum()

10Y      0
5Y       1
2Y     173
30Y    181
dtype: int64

In [7]:
monthly_data[monthly_data.isna().any(axis=1)]
#it seems like there is no data for the 30Y until 1977
#we will conduct analysis that has a timeframe capturing all the time values
#separate analysis only for the 10Y

,10Y,5Y,2Y,30Y
Date,,,,
1962-01-31,4.102,NaN,NaN,NaN
1962-02-28,4.002,3.944,NaN,NaN
1962-03-31,3.862,3.804,NaN,NaN
1962-04-30,3.862,3.654,NaN,NaN
1962-05-31,3.902,3.664,NaN,NaN
...,...,...,...,...
1976-09-30,7.552,6.604,6.34,NaN
1976-10-31,7.422,6.194,5.85,NaN
1976-11-30,7.012,5.964,5.82,NaN


In [8]:
monthly_data.dropna(inplace=True)

In [9]:
#getting to know the data..10Y Treasury was ~16% at some point during the our analyzed timeline
monthly_data.describe(percentiles=[0.1,0.5,0.9])

,10Y,5Y,2Y,30Y
count,542.000000,542.000000,542.000000,542.000000
mean,5.925097,5.499312,5.010875,6.314404
std,3.379906,3.623701,3.857816,3.075587
min,0.528200,0.204400,0.105400,1.192500
10%,1.915610,1.295270,0.418690,2.744600
50%,5.499500,5.164500,4.778200,5.810000
90%,10.789000,10.667300,9.926000,10.948500
max,15.842000,16.154000,16.699000,15.202000


In [10]:
monthly_absolute_change=monthly_data-monthly_data.shift(periods=1)

In [11]:
monthly_percent_change=monthly_data.pct_change()

In [12]:
#how does each move last month compare with historical data:
monthly_absolute_change.describe(percentiles=[0.1,0.5,0.9]).append(monthly_absolute_change.iloc[-1]).iloc[1:]
#nowhere near the max, but we also have to take into consideration that rates are 3% while the historical max was almost 16%

,10Y,5Y,2Y,30Y
mean,-0.010100,-0.007789,-0.006866,-0.009967
std,0.346317,0.417639,0.467609,0.308654
min,-1.880000,-3.020000,-3.870000,-1.900000
10%,-0.368400,-0.422000,-0.400000,-0.310000
50%,0.002000,-0.007500,0.000000,-0.014000
90%,0.360000,0.400000,0.410000,0.321000
max,1.590000,2.940000,3.010000,1.180000
2022-03-31 00:00:00,0.513000,0.742200,0.902200,0.286500


In [13]:
#how does each move last month compare with historical data:
monthly_percent_change.describe(percentiles=[0.1,0.5,0.9]).append(monthly_percent_change.iloc[-1]).iloc[1:]
#apparently its not the max (we'll see later what month is the max), but definitely among the biggest changes.

,10Y,5Y,2Y,30Y
mean,0.000759,0.004098,0.007756,-0.000558
std,0.076887,0.109770,0.138663,0.056309
min,-0.417116,-0.593950,-0.731106,-0.222702
10%,-0.074267,-0.098550,-0.122581,-0.056791
50%,0.000525,-0.001428,0.000000,-0.002024
90%,0.078606,0.107400,0.136803,0.058096
max,0.334343,0.744097,0.803993,0.346513
2022-03-31 00:00:00,0.281096,0.432089,0.629896,0.132571


In [18]:

fig=go.Figure()
fig.add_trace(go.Bar(x=monthly_absolute_change.sort_values(by='10Y').head(10).index,y=monthly_absolute_change['10Y'].sort_values().head(10),marker=dict(color='red'),name='top 10 declines'))
fig.add_trace(go.Bar(x=monthly_absolute_change.index,y=monthly_absolute_change['10Y'],marker=dict(color='blue'),name='all_other_data',visible='legendonly'))
fig.add_trace(go.Bar(x=monthly_absolute_change.sort_values(by='10Y').tail(10).index,y=monthly_absolute_change['10Y'].sort_values().tail(10),marker=dict(color='green'),name='top 10 increases'))

fig.update_xaxes(range=[monthly_absolute_change.index[0],monthly_absolute_change.index[-1]])
fig.update_layout(title='Biggest monthly absolute moves')
fig.show()

In [21]:

fig=go.Figure()
fig.add_trace(go.Bar(x=monthly_percent_change.sort_values(by='10Y').head(10).index,y=monthly_percent_change['10Y'].sort_values().head(10),marker=dict(color='red'),name='top 10 declines'))
fig.add_trace(go.Bar(x=monthly_percent_change.index,y=monthly_percent_change['10Y'],marker=dict(color='blue'),name='all_other_data',visible='legendonly'))
fig.add_trace(go.Bar(x=monthly_percent_change.sort_values(by='10Y').tail(10).index,y=monthly_percent_change['10Y'].sort_values().tail(10),marker=dict(color='green'),name='top 10 increases'))

fig.update_xaxes(range=[monthly_percent_change.index[0],monthly_percent_change.index[-1]])
fig.update_layout(title='Biggest monthly moves (%)')
fig.show()

In [121]:
#monthly_percent_change.sort_values(by=1).head(10)


DatetimeIndex(['2020-03-31', '2008-11-30', '2019-08-31', '2015-01-31',
               '2008-12-31', '2020-02-29', '2020-01-31', '2011-08-31',
               '2016-06-30', '2020-07-31'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [24]:

from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=4, cols=1,subplot_titles=("10Y", "5Y", "2Y", "30Y"))

for i in range(4):
  
  fig.add_trace(go.Bar(x=monthly_percent_change.iloc[:,i].sort_values().head(10).index,y=monthly_percent_change.iloc[:,i].sort_values().head(10),marker=dict(color='red'),name='top 10 declines'),row=i+1,col=1)
  fig.add_trace(go.Bar(x=monthly_percent_change.index,y=monthly_percent_change.iloc[:,i],marker=dict(color='blue'),name='all_other_data',visible='legendonly'),row=i+1,col=1)
  fig.add_trace(go.Bar(x=monthly_percent_change.iloc[:,i].sort_values().tail(10).index,y=monthly_percent_change.iloc[:,i].sort_values().sort_values().tail(10),marker=dict(color='green'),name='top 10 increases'),row=i+1,col=1)

#fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.update_layout(title='Biggest monthly moves (%)',height=600,width=1200)
fig.show()



In [26]:
fig = make_subplots(rows=4, cols=1,subplot_titles=("10Y", "5Y", "2Y", "30Y"))

for i in range(4):
  
  fig.add_trace(go.Bar(x=monthly_absolute_change.iloc[:,i].sort_values().head(10).index,y=monthly_absolute_change.iloc[:,i].sort_values().head(10),marker=dict(color='red'),name='top 10 declines'),row=i+1,col=1)
  fig.add_trace(go.Bar(x=monthly_absolute_change.index,y=monthly_absolute_change.iloc[:,i],marker=dict(color='blue'),name='all_other_data',visible='legendonly'),row=i+1,col=1)
  fig.add_trace(go.Bar(x=monthly_absolute_change.iloc[:,i].sort_values().tail(10).index,y=monthly_absolute_change.iloc[:,i].sort_values().sort_values().tail(10),marker=dict(color='green'),name='top 10 increases'),row=i+1,col=1)

#fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.update_layout(title='Biggest monthly absolute moves',height=600,width=1200)
fig.show()


In [147]:
monthly_percent_change.iloc[-1]#.iloc[-1].index

10Y     0.281096
5Y      0.432089
2Y      0.629896
30Y     0.132571
zero    0.000000
Name: 2022-03-31 00:00:00, dtype: float64

In [30]:
fig = make_subplots(rows=2, cols=1,subplot_titles=("Top 10 Monthly Declines", "Top 10 Monthly Increases"))

fig.add_trace(go.Bar(x0=0,dx=1,text=monthly_percent_change.sort_values(by='10Y').head(10).index.strftime("%m - %Y <br> %b"),y=monthly_percent_change['10Y'].sort_values().head(10),name='top 10 declines',marker=dict(color='crimson')),row=1,col=1)
fig.add_trace(go.Bar(x0=10,text=monthly_percent_change.iloc[-1].name.strftime("%m - %Y <br> %b"),y=[monthly_percent_change.iloc[-1]['10Y']],visible='legendonly',name='last monht',marker=dict(color='grey')),row=1,col=1)
fig.add_trace(go.Bar(x0=0,dx=1,text=monthly_percent_change.sort_values(by='10Y').tail(10).index.strftime("%m - %Y <br> %b"),y=monthly_percent_change['10Y'].sort_values().tail(10),name='top 10 increases',marker=dict(color='green')),row=2,col=1)
fig.add_trace(go.Bar(x0=10,text=monthly_percent_change.iloc[-1].name.strftime("%m - %Y <br> %b"),y=[monthly_percent_change.iloc[-1]['10Y']],visible='legendonly',name='last month',marker=dict(color='grey')),row=2,col=1)

fig.update_layout(title='Top 10 Monthly Performance Declines 10Y <br> <i>(from left to right)</i> ',height=600,width=1200)
#fig.update_layout(title='Biggest monthly moves (%)',)

fig.show()

In [47]:
#what is the probability of the rate move (both in % and absolute terms):
import scipy.stats as stats
#mean,std = monthly_data.describe()['Return'].loc[['mean','std']]
for security in monthly_percent_change.columns:
  mean_m,std_m=monthly_percent_change.describe().loc[['mean','std'],security]
  # Calculate a z-score from a provided mean and standard deviation
  print (f'Stats for the {security}')
  if monthly_percent_change[security].iloc[-1]>mean_m:
    print(f"{monthly_percent_change.iloc[-1].name.strftime('%b%y')} return should happen in every other {(1/(1-stats.norm.cdf(x=monthly_percent_change[security].iloc[-1],loc=mean_m,scale=std_m))):0.00f} months")
    print(f'probability to get better outcome are {(1-stats.norm.cdf(x=monthly_percent_change[security].iloc[-1],loc=mean_m,scale=std_m))*100:0.02f}')
  else:
    print(f"{monthly_percent_change.iloc[-1].name.strftime('%b%y')} return should happen in every other {(1/stats.norm.cdf(x=monthly_percent_change[security].iloc[-1],loc=mean_m,scale=std_m)):0.00f} months")
    print(f'probability to get better outcome are {stats.norm.cdf(x=monthly_percent_change[security].iloc[-1],loc=mean_m,scale=std_m)*100:0.02f}')
  print(f"z-score is {(monthly_percent_change[security].iloc[-1]-mean_m)/std_m:0.02f}")
  print(f'Difference between the average move and last month is:{(monthly_percent_change[security].iloc[-1]-mean_m)*100:0.02f}%')
  print('\n')

Stats for the 10Y
Mar22 return should happen in every other 7511 months
probability to get better outcome are 0.01
z-score is 3.65
Difference between the average move and last month is:28.03%


Stats for the 5Y
Mar22 return should happen in every other 20704 months
probability to get better outcome are 0.00
z-score is 3.90
Difference between the average move and last month is:42.80%


Stats for the 2Y
Mar22 return should happen in every other 276491 months
probability to get better outcome are 0.00
z-score is 4.49
Difference between the average move and last month is:62.21%


Stats for the 30Y
Mar22 return should happen in every other 111 months
probability to get better outcome are 0.90
z-score is 2.36
Difference between the average move and last month is:13.31%




In [51]:

for security in monthly_absolute_change.columns:
  mean_m,std_m=monthly_absolute_change.describe().loc[['mean','std'],security]
  # Calculate a z-score from a provided mean and standard deviation
  print (f'Stats for the {security}')
  if monthly_absolute_change[security].iloc[-1]>mean_m:
    print(f"{monthly_absolute_change.iloc[-1].name.strftime('%b%y')} return should happen in every other {(1/(1-stats.norm.cdf(x=monthly_absolute_change[security].iloc[-1],loc=mean_m,scale=std_m))):0.00f} months")
    print(f'probability to get better outcome are {(1-stats.norm.cdf(x=monthly_absolute_change[security].iloc[-1],loc=mean_m,scale=std_m))*100:0.02f}%')
  else:
    print(f"{monthly_absolute_change.iloc[-1].name.strftime('%b%y')} return should happen in every other {(1/stats.norm.cdf(x=monthly_absolute_change[security].iloc[-1],loc=mean_m,scale=std_m)):0.00f} months")
    print(f'probability to get better outcome are {stats.norm.cdf(x=monthly_absolute_change[security].iloc[-1],loc=mean_m,scale=std_m)*100:0.02f}')
  print(f"z-score is {(monthly_absolute_change[security].iloc[-1]-mean_m)/std_m:0.02f}")
  print(f'Difference between the average move and last month is:{(monthly_absolute_change[security].iloc[-1]-mean_m)*100:0.02f}%')
  print('\n')


Stats for the 10Y
Mar22 return should happen in every other 15 months
probability to get better outcome are 6.55%
z-score is 1.51
Difference between the average move and last month is:52.31%


Stats for the 5Y
Mar22 return should happen in every other 28 months
probability to get better outcome are 3.63%
z-score is 1.80
Difference between the average move and last month is:75.00%


Stats for the 2Y
Mar22 return should happen in every other 39 months
probability to get better outcome are 2.59%
z-score is 1.94
Difference between the average move and last month is:90.91%


Stats for the 30Y
Mar22 return should happen in every other 6 months
probability to get better outcome are 16.84%
z-score is 0.96
Difference between the average move and last month is:29.65%




In [54]:
quarterly_data=data.resample('Q').last()

In [55]:
quarterly_data.isna().sum()

10Y     0
5Y      0
2Y     57
30Y    60
dtype: int64

In [57]:
quarterly_data[quarterly_data.isna().any(axis=1)].head()

,10Y,5Y,2Y,30Y
Date,,,,
1962-03-31,3.862,3.804,NaN,NaN
1962-06-30,4.002,3.664,NaN,NaN
1962-09-30,3.942,3.664,NaN,NaN
1962-12-31,3.852,3.604,NaN,NaN
1963-03-31,3.952,3.684,NaN,NaN


In [58]:
quarterly_data.dropna(inplace=True)

In [59]:
quarterly_data.describe(percentiles=[0.1,0.5,0.9])

,10Y,5Y,2Y,30Y
count,181.000000,181.000000,181.000000,181.000000
mean,5.913457,5.504212,5.020020,6.299825
std,3.394299,3.645760,3.893172,3.092007
min,0.656100,0.276800,0.121100,1.321400
10%,1.917500,1.263000,0.422200,2.751800
50%,5.446000,5.102000,4.801000,5.875000
90%,10.622000,10.544000,10.019000,10.692000
max,15.842000,16.134000,16.539000,15.202000


In [60]:
quarterly_absolute_change=quarterly_data-quarterly_data.shift(periods=1)

In [61]:
quarterly_percent_change=quarterly_data.pct_change()

In [62]:
quarterly_absolute_change.describe(percentiles=[0.1,0.5,0.9]).append(quarterly_absolute_change.iloc[-1]).iloc[1:]

,10Y,5Y,2Y,30Y
mean,-0.028244,-0.024578,-0.020581,-0.029624
std,0.688522,0.826265,0.961134,0.618726
min,-2.710000,-4.420000,-6.530000,-2.380000
10%,-0.786200,-1.003280,-0.940010,-0.623400
50%,0.028900,0.044550,0.038800,0.003500
90%,0.693700,0.731310,0.843000,0.588030
max,2.310000,3.210000,3.830000,2.190000
2022-03-31 00:00:00,0.827900,1.196900,1.602300,0.544400


In [63]:
quarterly_percent_change.describe(percentiles=[0.1,0.5,0.9]).append(quarterly_percent_change.iloc[-1]).iloc[1:]

,10Y,5Y,2Y,30Y
mean,0.005692,0.018099,0.031094,-0.000297
std,0.155198,0.230410,0.294904,0.108999
min,-0.650847,-0.775340,-0.843541,-0.447020
10%,-0.138567,-0.171579,-0.219025,-0.105248
50%,0.012265,0.007162,0.019381,0.000596
90%,0.146423,0.167882,0.242142,0.109449
max,0.905826,1.603381,2.188337,0.465499
2022-03-31 00:00:00,0.548242,0.947664,2.188337,0.286045


In [67]:
fig=go.Figure()
fig.add_trace(go.Bar(x=quarterly_absolute_change.sort_values(by='10Y').head(10).index,y=quarterly_absolute_change['10Y'].sort_values().head(10),marker=dict(color='red'),name='top 10 declines'))
fig.add_trace(go.Bar(x=quarterly_absolute_change.index,y=quarterly_absolute_change['10Y'],marker=dict(color='blue'),name='all_other_data',visible='legendonly'))
fig.add_trace(go.Bar(x=quarterly_absolute_change.sort_values(by='10Y').tail(10).index,y=quarterly_absolute_change['10Y'].sort_values().tail(10),marker=dict(color='green'),name='top 10 increases'))
fig.update_xaxes(range=[quarterly_absolute_change.index[0],quarterly_absolute_change.index[-1]])
fig.update_layout(title='Biggest quarterly absolute moves for 10Y')
fig.show()


In [68]:
fig=go.Figure()
fig.add_trace(go.Bar(x=quarterly_percent_change.sort_values(by='10Y').head(10).index,y=quarterly_percent_change['10Y'].sort_values().head(10),marker=dict(color='red'),name='top 10 declines'))
fig.add_trace(go.Bar(x=quarterly_percent_change.index,y=quarterly_percent_change['10Y'],marker=dict(color='blue'),name='all_other_data',visible='legendonly'))
fig.add_trace(go.Bar(x=quarterly_percent_change.sort_values(by='10Y').tail(10).index,y=quarterly_percent_change['10Y'].sort_values().tail(10),marker=dict(color='green'),name='top 10 increases'))

fig.update_xaxes(range=[quarterly_percent_change.index[0],quarterly_percent_change.index[-1]])
fig.update_layout(title='Biggest quarterly moves for 10Y(%)')
fig.show()


In [69]:
fig = make_subplots(rows=4, cols=1,subplot_titles=("10Y", "5Y", "2Y", "30Y"))

for i in range(4):
  
  fig.add_trace(go.Bar(x=quarterly_percent_change.iloc[:,i].sort_values().head(10).index,y=quarterly_percent_change.iloc[:,i].sort_values().head(10),marker=dict(color='red'),name='top 10 declines'),row=i+1,col=1)
  fig.add_trace(go.Bar(x=quarterly_percent_change.index,y=quarterly_percent_change.iloc[:,i],marker=dict(color='blue'),name='all_other_data',visible='legendonly'),row=i+1,col=1)
  fig.add_trace(go.Bar(x=quarterly_percent_change.iloc[:,i].sort_values().tail(10).index,y=quarterly_percent_change.iloc[:,i].sort_values().sort_values().tail(10),marker=dict(color='green'),name='top 10 increases'),row=i+1,col=1)

#fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.update_layout(title='Biggest quarterly moves (%)',height=600,width=1200)
fig.show()


In [70]:
fig = make_subplots(rows=4, cols=1,subplot_titles=("10Y", "5Y", "2Y", "30Y"))

for i in range(4):
  
  fig.add_trace(go.Bar(x=quarterly_absolute_change.iloc[:,i].sort_values().head(10).index,y=quarterly_absolute_change.iloc[:,i].sort_values().head(10),marker=dict(color='red'),name='top 10 declines'),row=i+1,col=1)
  fig.add_trace(go.Bar(x=quarterly_absolute_change.index,y=quarterly_absolute_change.iloc[:,i],marker=dict(color='blue'),name='all_other_data',visible='legendonly'),row=i+1,col=1)
  fig.add_trace(go.Bar(x=quarterly_absolute_change.iloc[:,i].sort_values().tail(10).index,y=quarterly_absolute_change.iloc[:,i].sort_values().sort_values().tail(10),marker=dict(color='green'),name='top 10 increases'),row=i+1,col=1)

#fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.update_layout(title='Biggest quarterly absolute moves',height=600,width=1200)
fig.show()



In [78]:
fig = make_subplots(rows=2, cols=1,subplot_titles=("Top 10 Quarterly Declines", "Top 10 Quarterly Increases"))

fig.add_trace(go.Bar(x0=0,dx=1,text=quarterly_percent_change.sort_values(by='10Y').head(10).index.strftime("%m - %Y <br> %b"),y=quarterly_percent_change['10Y'].sort_values().head(10),name='top 10 declines',marker=dict(color='crimson')),row=1,col=1)
fig.add_trace(go.Bar(x0=10,text=quarterly_percent_change.iloc[-1].name.strftime("%m- %Y <br> %b"),y=[quarterly_percent_change.iloc[-1]['10Y']],visible='legendonly',name='last monht',marker=dict(color='grey')),row=1,col=1)
fig.add_trace(go.Bar(x0=0,dx=1,text=quarterly_percent_change.sort_values(by='10Y').tail(10).index.strftime("%m - %Y <br> %b"),y=quarterly_percent_change['10Y'].sort_values().tail(10),name='top 10 increases',marker=dict(color='green')),row=2,col=1)
fig.add_trace(go.Bar(x0=10,text=quarterly_percent_change.iloc[-1].name.strftime("%m- %Y <br> %b"),y=[quarterly_percent_change.iloc[-1]['10Y']],visible='legendonly',name='last month',marker=dict(color='grey')),row=2,col=1)

fig.update_layout(title='Top 10 Quarterly Performance Declines 10Y <br> <i>(from left to right)</i> ',height=600,width=1200)
#fig.update_layout(title='Biggest quarterly moves (%)',)

fig.show()


In [81]:
fig = make_subplots(rows=2, cols=1,subplot_titles=("Top 10 Quarterly Declines", "Top 10 Quarterly Increases"))

fig.add_trace(go.Bar(x0=0,dx=1,text=quarterly_absolute_change.sort_values(by='10Y').head(10).index.strftime("%m - %Y <br> %b"),y=quarterly_absolute_change['10Y'].sort_values().head(10),name='top 10 declines',marker=dict(color='crimson')),row=1,col=1)
fig.add_trace(go.Bar(x0=10,text=quarterly_absolute_change.iloc[-1].name.strftime("%m- %Y <br> %b"),y=[quarterly_absolute_change.iloc[-1]['10Y']],visible='legendonly',name='last monht',marker=dict(color='grey')),row=1,col=1)
fig.add_trace(go.Bar(x0=0,dx=1,text=quarterly_absolute_change.sort_values(by='10Y').tail(10).index.strftime("%m - %Y <br> %b"),y=quarterly_absolute_change['10Y'].sort_values().tail(10),name='top 10 increases',marker=dict(color='green')),row=2,col=1)
fig.add_trace(go.Bar(x0=10,text=quarterly_absolute_change.iloc[-1].name.strftime("%m- %Y <br> %b"),y=[quarterly_absolute_change.iloc[-1]['10Y']],visible='legendonly',name='last month',marker=dict(color='grey')),row=2,col=1)

fig.update_layout(title='Top 10 Quarterly Performance Declines 10Y <br> <i>(from left to right)</i> ',height=600,width=1200)
#fig.update_layout(title='Biggest quarterly moves (%)',)

fig.show()


In [74]:
for security in quarterly_percent_change.columns:
  mean_m,std_m=quarterly_percent_change.describe().loc[['mean','std'],security]
  # Calculate a z-score from a provided mean and standard deviation
  print (f'Stats for the {security}')
  if quarterly_percent_change[security].iloc[-1]>mean_m:
    print(f"{quarterly_percent_change.iloc[-1].name.strftime('%b%y')} return should happen in every other {(1/(1-stats.norm.cdf(x=quarterly_percent_change[security].iloc[-1],loc=mean_m,scale=std_m))):0.00f} months")
    print(f'probability to get better outcome are {(1-stats.norm.cdf(x=quarterly_percent_change[security].iloc[-1],loc=mean_m,scale=std_m))*100:0.02f}')
  else:
    print(f"{quarterly_percent_change.iloc[-1].name.strftime('%b%y')} return should happen in every other {(1/stats.norm.cdf(x=quarterly_percent_change[security].iloc[-1],loc=mean_m,scale=std_m)):0.00f} months")
    print(f'probability to get better outcome are {stats.norm.cdf(x=quarterly_percent_change[security].iloc[-1],loc=mean_m,scale=std_m)*100:0.02f}')
  print(f"z-score is {(quarterly_percent_change[security].iloc[-1]-mean_m)/std_m:0.02f}")
  print(f'Difference between the average move and last month is:{(quarterly_percent_change[security].iloc[-1]-mean_m)*100:0.02f}%')
  print('\n')



Stats for the 10Y
Mar22 return should happen in every other 4233 months
probability to get better outcome are 0.02
z-score is 3.50
Difference between the average move and last month is:54.25%


Stats for the 5Y
Mar22 return should happen in every other 36534 months
probability to get better outcome are 0.00
z-score is 4.03
Difference between the average move and last month is:92.96%


Stats for the 2Y
Mar22 return should happen in every other 7778237698395 months
probability to get better outcome are 0.00
z-score is 7.32
Difference between the average move and last month is:215.72%


Stats for the 30Y
Mar22 return should happen in every other 232 months
probability to get better outcome are 0.43
z-score is 2.63
Difference between the average move and last month is:28.63%




In [77]:
for security in quarterly_absolute_change.columns:
  mean_m,std_m=quarterly_absolute_change.describe().loc[['mean','std'],security]
  # Calculate a z-score from a provided mean and standard deviation
  print (f'Stats for the {security}')
  if quarterly_absolute_change[security].iloc[-1]>mean_m:
    print(f"{quarterly_absolute_change.iloc[-1].name.strftime('%b%y')} return should happen in every other {(1/(1-stats.norm.cdf(x=quarterly_absolute_change[security].iloc[-1],loc=mean_m,scale=std_m))):0.00f} quarters")
    print(f'probability to get better outcome are {(1-stats.norm.cdf(x=quarterly_absolute_change[security].iloc[-1],loc=mean_m,scale=std_m))*100:0.02f}%')
  else:
    print(f"{quarterly_absolute_change.iloc[-1].name.strftime('%b%y')} return should happen in every other {(1/stats.norm.cdf(x=quarterly_absolute_change[security].iloc[-1],loc=mean_m,scale=std_m)):0.00f} quarters")
    print(f'probability to get better outcome are {stats.norm.cdf(x=quarterly_absolute_change[security].iloc[-1],loc=mean_m,scale=std_m)*100:0.02f}')
  print(f"z-score is {(quarterly_absolute_change[security].iloc[-1]-mean_m)/std_m:0.02f}")
  print(f'Difference between the average move and last quarter is:{(quarterly_absolute_change[security].iloc[-1]-mean_m)*100:0.02f}%')
  print('\n')




Stats for the 10Y
Mar22 return should happen in every other 9 quarters
probability to get better outcome are 10.69%
z-score is 1.24
Difference between the average move and last quarter is:85.61%


Stats for the 5Y
Mar22 return should happen in every other 14 quarters
probability to get better outcome are 6.97%
z-score is 1.48
Difference between the average move and last quarter is:122.15%


Stats for the 2Y
Mar22 return should happen in every other 22 quarters
probability to get better outcome are 4.57%
z-score is 1.69
Difference between the average move and last quarter is:162.29%


Stats for the 30Y
Mar22 return should happen in every other 6 quarters
probability to get better outcome are 17.68%
z-score is 0.93
Difference between the average move and last quarter is:57.40%




In [19]:
print(f"{monthly_data.sort_values(by='Return').index[0].year} - {monthly_data.sort_values(by='Return').index[0].month}"),
round(monthly_data['Return'].sort_values().iloc[0],2)

In [ ]:
#quarterly performance
quarterly_data=data.resample('Q').last()
quarterly_data['Return']=quarterly_data['Price'].pct_change()*100
fig=go.Figure()
fig.add_trace(go.Bar(x=quarterly_data.sort_values(by='Return').head(10).index,y=quarterly_data['Return'].sort_values().head(10)))
fig.show()

In [ ]:
def own_probability(timeline,date):
  unique_data=data.resample(timeline).last()
  unique_data['Return']=unique_data['Price'].pct_change()*100

  o_mean,o_std = unique_data.describe()['Return'].loc[['mean','std']]

  if(unique_data['Return'].loc[date][0]>o_mean):    
    print(f"{unique_data.loc[date].index[0].strftime('%b%y')} return should happen in every other {(1/(1-stats.norm.cdf(x=unique_data['Return'].loc[date][0],loc=o_mean,scale=o_std))):0.00f} {timeline}")
    print(f"z-score is {(unique_data['Return'].loc[date][0]-o_mean)/o_std:0.02f}")
    print(f"Probability to have better outcome is {1-stats.norm.cdf(x=unique_data['Return'].loc[date][0],loc=o_mean,scale=o_std)}")
  else:
    print(f"{unique_data.loc[date].index[0].strftime('%b%y')} return should happen in every other {(1/stats.norm.cdf(x=unique_data['Return'].loc[date][0],loc=o_mean,scale=o_std)):0.00f} {timeline}")
    print(f"z-score is {(unique_data['Return'].loc[date][0]-o_mean)/o_std:0.02f}")
    print(f"Probability to have worse outcome is {stats.norm.cdf(x=unique_data['Return'].loc[date][0],loc=o_mean,scale=o_std)}")
  
own_probability('Y','2022')
